In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
from typing import Tuple

from AE_Rec_Sys import *
from consts import *
from utils import *

In [3]:
DATA_PATH = os.path.abspath(os.path.join(os.curdir, 'Data'))
train_df = pd.read_csv(os.path.abspath(os.path.join(DATA_PATH, 'Train.csv')), index_col=False)
test_rand_df = pd.read_csv(os.path.abspath(os.path.join(DATA_PATH, 'RandomTest.csv')))
test_pop_df = pd.read_csv(os.path.abspath(os.path.join(DATA_PATH, 'PopularityTest.csv')))

In [4]:
train_df.shape

(988129, 2)

In [ ]:
np.where(train_df)

In [16]:
train_df, test_rand_df, test_pop_df = read_data_files()
train_df, train_preferences_matrix, validation_rand_df = get_validation_and_train_matrix(train_df=train_df)

In [18]:
train_preferences_matrix.sum()

982089.0

In [19]:
train_df.shape

(982089, 2)

In [15]:
train_df.shape

(988129, 2)

In [ ]:
create_user_items_preferences_matrix(df=train_df)

In [5]:
train_df.sample(5)

,UserID,ItemID
385526,1368,3402
395052,614,318
243837,613,1268
937022,5028,2936
922625,3328,1264


In [6]:
get_train_random_validation_data(train_df)

(array([[1963, 2102],
        [1963, 1156],
        [1963, 2555],
        ...,
        [5201,  963],
        [5201, 1274],
        [5201, 2993]], dtype=int64),
 array([[   1, 1253, 2693],
        [   2,   36, 3051],
        [   3, 2162, 2541],
        ...,
        [6038, 1438, 1850],
        [6039, 2013, 2015],
        [6040, 2273,  314]], dtype=int64))

In [25]:
validation_df

,UserID,Item1,Item2
27,1963,2241,3283
159,5951,1939,161
216,1254,507,1444
242,2738,2793,1967
361,3527,557,1621
...,...,...,...
986635,4410,1878,2142
987605,2766,517,3490
987652,2380,1438,1078
987717,2465,1506,423


In [19]:
test_pop_df.sample(5)

,UserID,Item1,Item2
5615,4106,127,3354
2783,5699,1896,3373
1751,794,3238,2796
2672,1859,1807,1658
4130,2923,2183,1743


In [20]:
test_rand_df.sample(5)

,UserID,Item1,Item2
2324,3038,406,2790
5354,4644,3627,1017
178,4026,3435,3145
4935,4321,1461,1710
4952,2061,762,3009


In [4]:
print(f'Number of Users: {len(train_df.UserID.unique())}; Number of Items: {len(train_df.ItemID.unique())}')
print('One-Based (not starts from zero)')

Number of Users: 6040; Number of Items: 3705
One-Based (not starts from zero)


In [5]:
assert sorted(list(train_df.UserID.unique())) == list(range(1, len(train_df.UserID.unique()) + 1))

Items is missing with item 1750

In [6]:
train_df.UserID.max(), train_df.ItemID.max()

(6040, 3706)

In [7]:
def get_num_users_items(df):
    return df.UserID.max() + 1, df.ItemID.max()

In [8]:
NUM_USERS, NUM_ITEMS = train_df.UserID.max(), train_df.ItemID.max()

### UserID

In [9]:
train_df.UserID.value_counts(dropna=False, sort=True)

3772    2312
2736    1848
5719    1741
2741    1593
4084    1519
        ... 
3542      18
1063      18
1128      18
4046      18
4183      18
Name: UserID, Length: 6040, dtype: int64

### ItemID

In [10]:
train_df.ItemID.value_counts(dropna=False, sort=True)

406     3342
814     2938
127     2922
405     2821
3523    2616
        ... 
984        1
430        1
1497       1
2523       1
1382       1
Name: ItemID, Length: 3705, dtype: int64

### Create validation set - Negative sampling

#### Ramdom sampling

In [217]:
train_df, validation_df = get_train_random_validation_data(train_df)

In [219]:
validation_df_int = validation_df.astype(np.int64)

In [158]:
validation_df_int

NameError: name 'validation_df_int' is not defined

In [221]:
with h5py.File(PROJECT_PATH / 'hdf5' / 'validation_df_CHECK.h5', 'w') as hf:
    hf.create_dataset(obj_name, data=validation_df_int)

In [152]:
with h5py.File(PROJECT_PATH / 'hdf5' / 'validation_rand_df.h5', 'r') as hf:
     validation_rand_df = hf['validation_rand_df'][:]

In [118]:
np.hstack((a, validation_rand_df.bit.values.reshape(-1, 1)))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Tasks
* Export results to csv as format
    * Verify export works as expected  (csv)
    * bit classification in the right order
* Check HP:
    * Dropout?
* Draw probability validation of items

In [165]:
test_pop_df

,UserID,Item1,Item2
0,1963,1503,2798
1,5951,1098,1511
2,1254,366,3523
3,2738,1546,197
4,3527,406,3527
...,...,...,...
6035,4410,3142,3538
6036,2766,2016,963
6037,2380,3163,195
6038,2465,2993,982


In [ ]:
test_pop_df['bit'] = 